In [ ]:
#!pip install ray[rllib,tune]
#!pip install pettingzoo pygame pymunk
#!pip install torch

In [2]:
import ray
import glob
import sys

from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv, ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-29 01:48:39,790	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-29 01:48:41,507	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
import sys
from glob import glob
from os import path
from ray.rllib.policy.policy import Policy

ray.shutdown() 

# Trained to about 0 combined return
checkpoint_path = "/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
#specs = {path.basename(p) : SingleAgentRLModuleSpec(load_state_path=p) for p in pols} # Non-deterministic policy weight return (implies new)


num_agents = 2

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        #model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            #load_state_path=
            #module_specs=specs,
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)

resto_algo = resto_config.build()
resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()

2024-08-29 02:06:20,467	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.

In [7]:
resto_algo.evaluate()

{'env_runners': {'episode_reward_max': np.float64(104.48447046235398),
  'episode_reward_min': np.float64(-49.97758701259906),
  'episode_reward_mean': np.float64(34.31495169008534),
  'episode_len_mean': np.float64(500.0),
  'episode_media': {},
  'episodes_timesteps_total': 5000,
  'policy_reward_min': {'pursuer_0': np.float64(-27.80399612520721),
   'pursuer_1': np.float64(-74.73581968384396)},
  'policy_reward_max': {'pursuer_0': np.float64(108.50397211584374),
   'pursuer_1': np.float64(66.59594375510522)},
  'policy_reward_mean': {'pursuer_0': np.float64(32.97170784631929),
   'pursuer_1': np.float64(1.3432438437660394)},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [np.float64(44.487376281462424),
    np.float64(98.65380257932057),
    np.float64(84.50964155764572),
    np.float64(20.08502968193733),
    np.float64(104.48447046235398),
    np.float64(-38.577462399422544),
    np.float64(-49.97758701259906),
    np.float64(20.465628189378837),
    np.float64(91.2633

In [2]:
import sys
sys.argv = ['']

parser = add_rllib_example_script_args(
    default_iters=200,
    default_timesteps=1000000,
    default_reward=0.0,
)
args = parser.parse_args()

In [43]:
ray.shutdown() 

num_agents = 2

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}

base_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
)

algo = base_config.build()

2024-08-28 18:48:24,292	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/usr/local/lib/python3.10/dist-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/usr/local/lib/python3.10/dist-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.

In [29]:
#algo.export_policy_model("/root/scrap_outputs/")
#algo.export_model("model", "/root/scrap_outputs/")
algo.get_policy("pursuer_0").get_weights()

{'_logits._model.0.weight': array([[-0.00096801,  0.00098325,  0.00243963, ..., -0.0006884 ,
          0.01252443,  0.00090427],
        [-0.00969569,  0.00104609, -0.00021788, ..., -0.00139742,
          0.00206522, -0.00116132],
        [-0.00290761, -0.00024362, -0.00082618, ..., -0.00579849,
          0.00045324, -0.00129697],
        [-0.0030074 ,  0.00228428, -0.00139327, ..., -0.00469326,
         -0.01144298, -0.00017425]], dtype=float32),
 '_logits._model.0.bias': array([ 0.0004101 , -0.00077841, -0.00017921, -0.0007516 ], dtype=float32),
 '_hidden_layers.0._model.0.weight': array([[-0.1043953 , -0.03646389, -0.03957004, ..., -0.01649845,
          0.02555791,  0.02971697],
        [-0.07682262,  0.0685934 , -0.03567628, ..., -0.07106235,
          0.0290572 ,  0.0388599 ],
        [-0.02280383,  0.04673399, -0.01213942, ..., -0.1036943 ,
          0.00324975, -0.05494868],
        ...,
        [ 0.02286639, -0.03893814, -0.00669495, ...,  0.01313722,
         -0.0142059 ,  0.

In [22]:
algo.step()

2024-08-28 17:54:31,536	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0),
     'grad_gnorm': np.float32(1.9571484),
     'cur_kl_coeff': np.float64(0.20000000000000004),
     'cur_lr': np.float64(5.0000000000000016e-05),
     'total_loss': np.float64(9.20222436885039),
     'policy_loss': np.float64(-0.007261630720555938),
     'vf_loss': np.float64(9.208502007524173),
     'vf_explained_var': np.float64(3.984582920869191e-05),
     'kl': np.float64(0.004919840696008274),
     'entropy': np.float64(2.7816437045733133),
     'entropy_coeff': np.float64(0.0)},
    'model': {},
    'custom_metrics': {},
    'num_agent_steps_trained': np.float64(125.0),
    'num_grad_updates_lifetime': np.float64(480.5),
    'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)},
   'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0),
     'grad_gnorm': np.float32(1.4359734),
     'cur_kl_coeff': np.float64(0.20000000000

In [38]:
blank_pol = SingleAgentRLModuleSpec()
print(blank_pol)

from ray.rllib.algorithms.ppo.torch.ppo_torch_rl_module import PPOTorchRLModule

renewed_pol = SingleAgentRLModuleSpec(
    #module_class=PPOTorchRLModule,
    #load_state_path="/root/scrap_outputs/"
    load_state_path="/tmp/tmpmnsw4z8d/policies/pursuer_0"
)
renewed_pol.build()
print(renewed_pol)

SingleAgentRLModuleSpec(module_class=None, observation_space=None, action_space=None, inference_only=False, model_config_dict=None, catalog_class=None, load_state_path=None)


ValueError: RLModule class is not set.

In [36]:
save_result = algo.save()
path_to_checkpoint = save_result.checkpoint.path

In [37]:
path_to_checkpoint

'/tmp/tmpmnsw4z8d'

In [40]:
from ray.rllib.policy.policy import Policy

Policy.from_checkpoint("/tmp/tmpmnsw4z8d/policies/pursuer_0")

PPOTorchPolicy

In [49]:
from ray.rllib.utils.test_utils import run_rllib_example_script_experiment 
import sys
sys.argv = ['']

parser = add_rllib_example_script_args(
    default_iters=5,
    default_timesteps=1000000,
    default_reward=0.0,
    
)
args = parser.parse_args()

args.num_agents = 2
args.checkpoint_at_end = True

register_env(f"{args.num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=args.num_agents)))

ray.shutdown()
exp_results = run_rllib_example_script_experiment(base_config, args)

2024-08-28 19:16:37,825	WARNING services.py:2017 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 66924544 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-08-28 19:16:37,941	INFO worker.py:1781 -- Started a local Ray instance.
2024-08-28 19:16:38,680	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
2024-08-28 19:16:38,682	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `

== Status ==
Current time: 2024-08-28 19:16:38 (running for 00:00:00.15)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(PPO pid=784497) 2024-08-28 19:16:42,110	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.


== Status ==
Current time: 2024-08-28 19:16:43 (running for 00:00:05.17)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(PPO pid=784497) 2024-08-28 19:16:47,106	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
(PPO pid=784497) Install gputil for GPU system monitoring.


...

Trial name,agent_timesteps_total,counters,custom_metrics,episode_media,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_2_agent_env_0cc56_00000,40000,"{'num_env_steps_sampled': 20000, 'num_env_steps_trained': 20000, 'num_agent_steps_sampled': 40000, 'num_agent_steps_trained': 40000}",{},{},"{'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(1.8481598), 'cur_kl_coeff': np.float64(0.05000000000000001), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(4.906999621043602), 'policy_loss': np.float64(-0.016920431756201046), 'vf_loss': np.float64(4.9233541076382), 'vf_explained_var': np.float64(-0.0003006997828682264), 'kl': np.float64(0.011318726324657297), 'entropy': np.float64(2.8527599732081095), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(4320.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}, 'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.250387), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(8.95439296066761), 'policy_loss': np.float64(-0.017483378629064343), 'vf_loss': np.float64(8.970676642656326), 'vf_explained_var': np.float64(6.644055247306824e-07), 'kl': np.float64(0.011997064870593026), 'entropy': np.float64(2.6846034176647664), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(4320.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}}, 'num_env_steps_sampled': 20000, 'num_env_steps_trained': 20000, 'num_agent_steps_sampled': 40000, 'num_agent_steps_trained': 40000}",40000,40000,40000,20000,20000,4000,190.559,20000,4000,190.559,2,0,0,4000,"{'cpu_util_percent': np.float64(4.956666666666666), 'ram_util_percent': np.float64(9.0)}","{'training_iteration_time_ms': 21023.473, 'restore_workers_time_ms': 0.035, 'training_step_time_ms': 21023.349, 'sample_time_ms': 7580.096, 'learn_time_ms': 13429.353, 'learn_throughput': 297.855, 'synch_weights_time_ms': 12.497}"


== Status ==
Current time: 2024-08-28 19:17:09 (running for 00:00:30.40)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-08-28 19:17:14 (running for 00:00:35.42)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-08-28 19:17:19 (running for 00:00:40.44)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time

(PPO pid=784497) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/PPO_2024-08-28_19-16-38/PPO_2_agent_env_0cc56_00000_0_2024-08-28_19-16-38/checkpoint_000000)
2024-08-28 19:18:32,518	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/PPO_2024-08-28_19-16-38' in 0.0154s.


== Status ==
Current time: 2024-08-28 19:18:32 (running for 00:01:53.84)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-08-28_19-16-36_886231_720728/artifacts/2024-08-28_19-16-38/PPO_2024-08-28_19-16-38/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)
+-----------------------------+------------+----------------------+--------+------------------+-------+-------------------+
| Trial name                  | status     | loc                  |   iter |   total time (s) |    ts |   combined return |
|-----------------------------+------------+----------------------+--------+------------------+-------+-------------------|
| PPO_2_agent_env_0cc56_00000 | TERMINATED | 10.42.142.216:784497 |      5 |          105.155 | 20000 |          -250.453 |
+-----------------------------+------------+----------------------+--------+------------------+-------+-------------------+




2024-08-28 19:18:32,735	INFO tune.py:1041 -- Total run time: 114.07 seconds (113.83 seconds for the tuning loop).


In [46]:
exp_results

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.2510386), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(4.503196743130684), 'policy_loss': np.float64(-0.012351240065375654), 'vf_loss': np.float64(4.514552159110705), 'vf_explained_var': np.float64(-4.657854636510213e-07), 'kl': np.float64(0.00995810289178584), 'entropy': np.float64(2.6592838952938718), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(4320.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}, 'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.5077887), 'cur_kl_coeff': np.float64(0.20000000000000004), 'cur_lr': np.float64(5.0000000000000016

In [50]:
#exp_results.save("/tmp/test")
exp_results.get_best_result()

Result(
  metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(1.8481598), 'cur_kl_coeff': np.float64(0.05000000000000001), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(4.906999621043602), 'policy_loss': np.float64(-0.016920431756201046), 'vf_loss': np.float64(4.9233541076382), 'vf_explained_var': np.float64(-0.0003006997828682264), 'kl': np.float64(0.011318726324657297), 'entropy': np.float64(2.8527599732081095), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(4320.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}, 'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.250387), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss'

Finally, a proper functioning path containing usable policy objects:
```
path=/root/ray_results/PPO_2024-08-28_19-16-38/PPO_2_agent_env_0cc56_00000_0_2024-08-28_19-16-38/checkpoint_000000
```

In [2]:
from glob import glob
from os import path
from ray.rllib.policy.policy import Policy

checkpoint_path = "/root/ray_results/PPO_2024-08-28_19-16-38/PPO_2_agent_env_0cc56_00000_0_2024-08-28_19-16-38/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
{path.basename(p) : Policy.from_checkpoint(p) for p in pols}


ModuleNotFoundError: No module named 'numpy._core'

In [65]:
algo.get_policy("pursuer_0").get_weights()

{'_logits._model.0.weight': array([[ 4.2849869e-04,  7.1076828e-04, -4.6757600e-06, ...,
          4.9453106e-04,  5.0544657e-04, -4.6216308e-05],
        [-6.1395345e-04,  1.8156106e-04,  3.8302711e-05, ...,
         -9.1722631e-04, -3.6863267e-04,  2.4935571e-04],
        [-7.5717538e-04,  8.4328500e-04,  5.7490711e-04, ...,
         -9.1318227e-04,  7.3682319e-04, -1.8278812e-04],
        [-1.5335288e-03, -4.4104998e-04,  3.1705943e-04, ...,
          2.5113211e-05, -6.6908996e-04,  5.0960114e-04]], dtype=float32),
 '_logits._model.0.bias': array([0., 0., 0., 0.], dtype=float32),
 '_hidden_layers.0._model.0.weight': array([[ 0.03287322,  0.08297306, -0.06368764, ...,  0.01571757,
          0.07756767,  0.03683888],
        [ 0.04736435, -0.02817171, -0.04709467, ...,  0.01066606,
          0.1087309 ,  0.05078119],
        [-0.01282683, -0.04872371, -0.04340782, ...,  0.10504086,
          0.03968241, -0.01037231],
        ...,
        [ 0.0444776 , -0.04372583,  0.04927925, ..., -0

In [4]:
resto_pursuer_0 = Policy.from_checkpoint("/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000/policies/pursuer_0")
# expected 16 args max, got 18 (pickle issue)

#resto_pursuer_0 = Policy.from_checkpoint("/root/ray_results/PPO_2024-08-28_19-16-38/PPO_2_agent_env_0cc56_00000_0_2024-08-28_19-16-38/checkpoint_000000/policies/pursuer_0")


resto_pursuer_0.get_weights()

{'_logits._model.0.weight': array([[-0.0057713 ,  0.02130763, -0.00332906, ..., -0.00251019,
          0.01308718, -0.01480941],
        [ 0.02027893, -0.00918621,  0.0372359 , ...,  0.00163348,
          0.01085026,  0.00847802],
        [-0.03894725,  0.07293524, -0.01994372, ...,  0.0666676 ,
         -0.03469629, -0.02116322],
        [ 0.00803491,  0.02462115, -0.01157856, ..., -0.0220483 ,
         -0.02909413,  0.0477805 ]], dtype=float32),
 '_logits._model.0.bias': array([ 0.00049785,  0.00283682, -0.01044037, -0.01154366], dtype=float32),
 '_hidden_layers.0._model.0.weight': array([[-0.06341984, -0.01866287,  0.04820696, ..., -0.06329054,
         -0.0094757 , -0.1880102 ],
        [ 0.04908092, -0.02408476,  0.01319859, ...,  0.0380294 ,
          0.11712942,  0.10649226],
        [ 0.00440866, -0.02424742, -0.0211874 , ..., -0.00685773,
          0.04866119, -0.04014146],
        ...,
        [ 0.05629121,  0.01958698, -0.01932536, ..., -0.06350598,
          0.00073303,  0.